In [11]:
import sklearn
import numpy as np
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt

In [5]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, cache = False)